In [94]:
import pandas as pd

In [95]:
df_ori = pd.read_csv('./01_raw_data/charger_info.csv', dtype={'year': 'Int64', 'zcode': 'Int64'})

In [96]:
df = df_ori.copy()
slow_types = {'02', '07', '08'}
df['charge_speed'] = df['chgerType'].astype(str).str.zfill(2).apply(
    lambda x: '완속' if x in slow_types else '급속'
)
region_yearly = (
    df.dropna(subset=['year', 'zcode'])
      .groupby(['zcode', 'year', 'charge_speed'])
      .size()
      .reset_index(name='count')
)
region_yearly_wide = (
    region_yearly
    .pivot(index=['zcode', 'year'], columns='charge_speed', values='count')
    .reset_index()
    .fillna(0)
)
region_yearly_wide.columns.name = None
zcode_abbr = {
    11: '서울', 26: '부산', 27: '대구', 28: '인천', 29: '광주',
    30: '대전', 31: '울산', 36: '세종', 41: '경기', 51: '강원',
    43: '충북', 44: '충남', 52: '전북', 46: '전남', 47: '경북',
    48: '경남', 50: '제주'
}
region_yearly_wide['sido'] = (
    region_yearly_wide['zcode']
    .astype(int)
    .map(zcode_abbr)
)
nationwide = (
    region_yearly_wide
    .groupby('year')[['완속', '급속']]
    .sum()
    .reset_index()
)

nationwide['sido'] = '전국'

region_yearly_wide = pd.concat([region_yearly_wide, nationwide], ignore_index=True)

region_yearly_wide = region_yearly_wide.sort_values(['year', 'sido']).reset_index(drop=True)


region_yearly_wide.drop(columns=['zcode'], inplace=True)

region_yearly_wide = region_yearly_wide[['year', 'sido', '완속', '급속']]
region_yearly_wide = region_yearly_wide[
    (region_yearly_wide['year'] >= 2019) & (region_yearly_wide['year'] <= 2024)
].reset_index(drop=True)

region_yearly_wide.head(50)


,year,sido,완속,급속
0,2019,강원,226.0,146.0
1,2019,경기,2287.0,568.0
2,2019,경남,781.0,164.0
3,2019,경북,478.0,232.0
4,2019,광주,390.0,66.0
5,2019,대구,506.0,105.0
6,2019,대전,373.0,77.0
7,2019,부산,846.0,62.0
8,2019,서울,1016.0,195.0
9,2019,세종,92.0,25.0


In [97]:

def yearly_to_monthly(df, round_to_int=False):
    monthly_rows = []
    
    for _, row in df.iterrows():
        year = int(row['year'])
        region = row['region']
        slow_yearly = int(row['slow'])
        fast_yearly = int(row['fast'])

        slow_per_month = slow_yearly / 12
        fast_per_month = fast_yearly / 12
        
        for month in range(1, 13):
            monthly_rows.append({
                'year': year,
                'month': month,
                'region': region,
                'slow_chargers_new': slow_per_month,
                'fast_chargers_new': fast_per_month,
            })
    
    monthly_df = pd.DataFrame(monthly_rows)

    if round_to_int:
        monthly_df['slow_chargers_new'] = monthly_df['slow_chargers_new'].round().astype(int)
        monthly_df['fast_chargers_new'] = monthly_df['fast_chargers_new'].round().astype(int)

    monthly_df['chargers_new'] = monthly_df['slow_chargers_new'] + monthly_df['fast_chargers_new']

    monthly_df['slow_chargers_cum'] = monthly_df.groupby('region')['slow_chargers_new'].cumsum()
    monthly_df['fast_chargers_cum'] = monthly_df.groupby('region')['fast_chargers_new'].cumsum()
    monthly_df['chargers_cum'] = monthly_df.groupby('region')['chargers_new'].cumsum()

    monthly_df = monthly_df.sort_values(['region', 'year', 'month']).reset_index(drop=True)
    return monthly_df

region_yearly_wide_eng = region_yearly_wide.rename(columns={
    '완속': 'slow',
    '급속': 'fast',
    'sido': 'region'
})[['year','region','slow','fast']]
region_monthly = yearly_to_monthly(region_yearly_wide_eng, round_to_int=False)

region_monthly.head(50)


,year,month,region,slow_chargers_new,fast_chargers_new,chargers_new,slow_chargers_cum,fast_chargers_cum,chargers_cum
0,2019,1,강원,18.833333,12.166667,31.000000,18.833333,12.166667,31.000000
1,2019,2,강원,18.833333,12.166667,31.000000,37.666667,24.333333,62.000000
2,2019,3,강원,18.833333,12.166667,31.000000,56.500000,36.500000,93.000000
3,2019,4,강원,18.833333,12.166667,31.000000,75.333333,48.666667,124.000000
4,2019,5,강원,18.833333,12.166667,31.000000,94.166667,60.833333,155.000000
5,2019,6,강원,18.833333,12.166667,31.000000,113.000000,73.000000,186.000000
6,2019,7,강원,18.833333,12.166667,31.000000,131.833333,85.166667,217.000000
7,2019,8,강원,18.833333,12.166667,31.000000,150.666667,97.333333,248.000000
8,2019,9,강원,18.833333,12.166667,31.000000,169.500000,109.500000,279.000000
9,2019,10,강원,18.833333,12.166667,31.000000,188.333333,121.666667,310.000000


In [98]:
region_monthly.to_csv('./02_interim_data/region_monthly_chargers.csv', index=False)